## В данном блокноте содержится pipeline для тонкой настройки Gemma 3 1B используя подход DoRA, рассмотренный в работе "DoRA: Weight-Decomposed Low-Rank Adaptation", [ICML](https://openreview.net/forum?id=3d5CIRG1n2) 2024.
 
- [archive](https://arxiv.org/html/2402.09353v6) 
- [github](https://github.com/NVlabs/DoRA)

PS: `index_Gemma1B.ipynb` **не актуален**. Процесс будет изложен в данном блокноте.

---

1. В работе исследуется применение методов PEFT, для моделей разных размеров (от 7B до 13B) и тонкой настройки на соответствующих датасетах (приведены в работе).
Сравнение полученных результатов (таблица 1 в работе). В работе утвердается, что при использовании метода DoRA, точность модели достигается как при полной тонкой настройки, но естественно осущестлвяется эффективней, так как обновляются только адаптеры, а не все параметры, как при тонкой настройки, что подтвержается результами сравнения, приведенными в таблице 1.

2. Я отойду от работы и сделаю изменения, что буду рассматривать. 
Как и в работе я проведу сравнение, только двух методов PEFT - LoRA и DoRA.
В качестве модели я использую казуальную модель (Как и Ламы в работе) [Gemma-3 1B](https://huggingface.co/google/gemma-3-1b-it) (параметров значительно меньше), выбор чисто из соображений к вычислительным ресурсам (запросто можно влять любую другую модель и проделать то же самое).

- В качестве решаемой задачи я выберу - ответ на вопрос по контексту.
- В качестве датасета и бенчмарка использую **SQuAD 2.0** (тренировочную и валидационную части).
- Выполню предобработку датасета для тонкой настройки (см. PSS2)
- В качестве метрик использую **Exact Match** и **F1**.
- Сделаю тонкую настройку на одной эпохе (чисто из прагматических соображений)

3. FT
* Проведу оценку модели на валидационной части **SQuAD 2.0** с разными промптами, температурами, приведу логи, сделаю анализ, какой промпт и температуру использовать для FT.
* Выполню PEFT с **LoRA**, оценю на валидационной части.
* Выполню PEFT c **DoRA**, оценю на валидационной части.
* Сравню результаты. Сделаю выводы. 



**Модель загружаю с Hugging Face (необходимо получить доступ к репозиторию, модель закрыта)**

**SQuAD 2.0 так же загружаю из HF**

---
- PSS: Предобработка датасета выполняется в `index_dataset_preprocess.ipynb`, в текущем блокноте только загружается
- PSS2: В ходе работы возникла проблема, что обрабатывать последовательности на 1024 токена не получается (на этапе обучение, на инференсе все нормально), следовательно, для тонкой настройки, тренировочную часть датасета (контекст самая большая часть) обрежу, валидационную часть так же обрежу. После сделаю финальный замер метрик на (1024 токенах) (да, модель будет обучаться на последовательностях меньшей длинны, чем изначально производилась оценка, на последовательностях меньшей длинны, скорей всего модель должна показать лучие резульаты, но финальные метрик (на 1024 токена) могут упасть, потому что модель не училась на таких больших последовательностях). Плюс стоит отметить, что в **SQuAD 2.0** последовательностей ближе к 1024 не так то и много, в среднем 300 токенов должно хватить, чтобы покрыть большую часть датасета, примерная оценка на глаз, полученная из анализа датасета.

### Необходимые импорты

In [1]:
import re
import string
import os
import sys

from datetime import datetime

import numpy as np

import torch
from torch.utils.data import DataLoader
from torch.cuda.amp import autocast, GradScaler
from torch.optim import AdamW
from torch.nn.functional import scaled_dot_product_attention

from transformers import AutoTokenizer, AutoModelForCausalLM

# for use LoRa (HF)
# from peft import LoraConfig, get_peft_model

# For use Dora
sys.path.append(os.path.join(os.getcwd(), "peft/src/"))
from peft import (  # noqa: E402
    LoraConfig,
    PeftModel,
    DoraConfig,
    BottleneckConfig,
    PrefixTuningConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
    set_peft_model_state_dict,
)


from datasets import load_dataset, load_from_disk #Загрузка датасета 
import evaluate #Для оценки модели

from tqdm import tqdm #for visualization

import gc

import logging

In [2]:
# ----- | Параметры | -----

OUT_DATA_CACHE = "../ft_v1/prep_datasets" #Директория, с предобработанными датасетами
CACHE_DIR = "../myDoRA_repeat/cache_dir" #Директория с кешем модели
EVAL_MODEL_DIR_OUT = "./eval_models_out" #Для вывода логов

MODEL_PATH = "google/gemma-3-1b-it" #Название модели

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu" #Проверка доступности CUDA

MAX_LEN_PROMPT_TOKENIZER = 256 #Длинна обрабатываемой последовательности
MAX_LEN_LABELS_AND_NEW_TOKENS = 16 #Длинна генерируемой последовательности
 
BATCH_SIZE = 2        #Размер батча
BATCH_SIZE_EVAL = 2    #Размер батча для оценки модели

TEMPERATURE = 0.7

PADDING_SIDE_TOKENIZER = "left" #Добавление отступов для tokenizer

PATH2DATA_TRAIN = f"{OUT_DATA_CACHE}/train_ds_short"
PATH2DATA_VAL = f"{OUT_DATA_CACHE}/val_ds_short"

# -------- Параметры для FT --------
EPOCH = 1
LEARNING_RATE = 1e-5
GRADIENT_ACCUMULATION_STEPS = 1 #Сколько батчей накапливать градиенты
TRAIN_LOGG_STEP = 1000
SAVE_MODEL_DIR_LORA = "./lora_ft_squad2"
SAVE_MODEL_DIR_DORA = "./dora_ft_squad2"

# -------- Параметры optimizer --------
BETAS = (0.9, 0.99)
EPS = 1e-8


print(
    f"model: {MODEL_PATH} ",
    f"DEVICE: {DEVICE}",
    f"train data: {PATH2DATA_TRAIN}",
    f"val data: {PATH2DATA_VAL}",
    f"batch_size: {BATCH_SIZE}",
    sep="\n"
)

model: google/gemma-3-1b-it 
DEVICE: cuda:0
train data: ../ft_v1/prep_datasets/train_ds_short
val data: ../ft_v1/prep_datasets/val_ds_short
batch_size: 2


### Модель Gemma-3 1B

In [3]:
torch.cuda.empty_cache()
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    cache_dir=CACHE_DIR,
    torch_dtype=torch.bfloat16,
    attn_implementation="eager" #Для обучения "eager" - более стабильный (стандартное внимание), для инференса или настройки с длинной последовательностью - "sdpa" (Flash Attention)
)
model.config.use_cache = False
model.to(DEVICE) 

Gemma3ForCausalLM(
  (model): Gemma3TextModel(
    (embed_tokens): Gemma3TextScaledWordEmbedding(262144, 1152, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma3DecoderLayer(
        (self_attn): Gemma3Attention(
          (q_proj): Linear(in_features=1152, out_features=1024, bias=False)
          (k_proj): Linear(in_features=1152, out_features=256, bias=False)
          (v_proj): Linear(in_features=1152, out_features=256, bias=False)
          (o_proj): Linear(in_features=1024, out_features=1152, bias=False)
          (q_norm): Gemma3RMSNorm((256,), eps=1e-06)
          (k_norm): Gemma3RMSNorm((256,), eps=1e-06)
        )
        (mlp): Gemma3MLP(
          (gate_proj): Linear(in_features=1152, out_features=6912, bias=False)
          (up_proj): Linear(in_features=1152, out_features=6912, bias=False)
          (down_proj): Linear(in_features=6912, out_features=1152, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma3RMSNorm((11

### Tokenizer

In [4]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, cache_dir=CACHE_DIR)
tokenizer.pad_token_id = 0 #Устанавливаем токен для отступа (Используется для добавления до максимальной длинны)
tokenizer.padding_side = PADDING_SIDE_TOKENIZER #Добавлять до максимальной длинны справа

### Dataset - SQuAD 2.0 (Предобработанный)

In [5]:
#Загружаем датасет
#Предобработанный датасет (256), но еще не токенизированный
train_dataset = load_from_disk(f"{OUT_DATA_CACHE}/train_ds_short")
val_dataset = load_from_disk(f"{OUT_DATA_CACHE}/val_ds_short")

small_train_ds = train_dataset.select(range(32))
small_val_ds = val_dataset.select(range(32))

In [6]:
print(
    len(train_dataset),
    len(val_dataset),
    len(small_train_ds),
    len(small_val_ds)
)

130105 11859 32 32


### Шаблон промпта и для предобработки датасета

In [7]:
def prompt_template(context, question, answer=None):
    #Первая инструкция, занимает 24 токена
    # instruction = "Answer the following question using only the information present in the provided context. Do not generate answers from outside the context."
    # instruction = "Answer the question based *only* on the context provided. If the answer cannot be found in the context, don't answer."
    # instruction = 'You are a strict AI assistant designed to answer a question based on context. Answer verbatim from the context. Do not guess or formulate, answer as it is in the context. If there is no answer in the context, answer "No answer".'
    # instruction = 'You are a strict AI assistant designed to generate answers based on a given context. Your task is to generate the answer exactly as it appears in the context, using the shortest possible phrase that matches the context verbatim. Do not rephrase, add extra text, or guess. If the question asks for something not explicitly stated in the context or involves negation (e.g., "What is not..."), return "No answer" unless the context explicitly provides a negative statement.'
    instruction = 'You are a strict AI assistant designed to answer a question based on context. Answer verbatim from the context. Do not guess or formulate, answer as it is in the context. If there is no answer in the context, answer "No answer".'
    if answer: #Если передали ответ
        return f"""{instruction}\nContext:\n{context}\nQuestion:\n{question}\nAnswer: {answer}"""
    return f"""{instruction}\nContext:\n{context}\nQuestion:\n{question}\nAnswer: """

def preprocess_datasets(examples):
    """
    Принимает батч   
    examples - словарь

    return - словарь (список, список, список)
    """
    contexts = examples["context"] #Берем батч контекстов
    questions = examples["question"] #Беремем батч вопросов
    # --- Без предобработки ---
    # answers = [ans["text"][0] if ans["text"] else "No answer" for ans in examples["answers"]] #Извлекаем ответы на естественном языке
    # --- С предобработкой ---
    answers = ["" if ans == "" else ans for ans in examples["answers"]] #Извлекаем ответы на естественном языке

    # Форматируем промпты
    prompts = [
        prompt_template(context, question) for context, question in zip(contexts, questions)
    ]
    
    #Можно сделать более лучше, но пока оставим как есть
    answers_prompts = [
        prompt_template(context, question, answer) for context, question, answer in zip(contexts, questions, answers)
    ]

    #Токенизируем промпты `prompts`. 
    inputs = tokenizer(
        prompts,
        padding="max_length",           #Дополняем последовательность до максимальной длинны
        truncation=True,                #Обрезать слишком длинные последовательности
        max_length=MAX_LEN_PROMPT_TOKENIZER,   #Максимальная длинна 
        return_tensors="pt"             #Возвращать как torch.tensor
    ) #Кстати возвращает не тензоры, а списки (словарь списков)
    
    with tokenizer.as_target_tokenizer(): #Обеспечивает корректную токенизацию меток
        labels = tokenizer(
            answers_prompts,
            padding="max_length",           #Дополняем последовательность до максимальной длинны
            truncation=True,                #Обрезать слишком длинные последовательности
            max_length=MAX_LEN_PROMPT_TOKENIZER,   #Максимальная длинна 
            return_tensors="pt"
        ) #Кстати возвращает не тензоры, а списки (словарь списков)

    inputs["labels"] = labels['input_ids'] #Добавляем в словарь с тензорами еще и labels
    inputs["answers"] = answers #Добавляем ответы на естественном языке
    
    #Поставим, чтобы просто игнорировал - tokenizer.pad_token_id и все
    # inputs["labels"][inputs["labels"] == tokenizer.pad_token_id] = -100 #Для того, чтобы токены отступа игнорировались
    return inputs

#### Формируем промпты и токенизируем

In [8]:
# Используем весь датасет, по контексту не усекаем (все влезает)
train_ds_tokenize = train_dataset.map(
    preprocess_datasets, #Функция, которая применяется ко всем строкам
    batched=True,        #Использовать батчинг
    num_proc=1,         #Количество процессов
    remove_columns=train_dataset.column_names,  #Удаляем исходные колонки
    cache_file_name="./cdatasets/train_ds_full_1024.cache" #Папка с кешом предобработанных данных
)

val_ds_tokenize = val_dataset.map(
    preprocess_datasets, #Функция, которая применяется ко всем строкам
    batched=True,        #Использовать батчинг
    num_proc=1,         #Количество процессов
    remove_columns=val_dataset.column_names,  #Удаляем исходные колонки
    cache_file_name="./cdatasets/val_ds_full_1024.cache" #Папка с кешом предобработанных данных
)


small_train_ds_tokenize = small_train_ds.map(
    preprocess_datasets, #Функция, которая применяется ко всем строкам
    batched=True,        #Использовать батчинг
    num_proc=1,         #Количество процессов
    remove_columns=small_train_ds.column_names,  #Удаляем исходные колонки
    cache_file_name="./cdatasets/train_ds_short_1024.cache" #Папка с кешом предобработанных данных
)

small_val_ds_tokenize = small_val_ds.map(
    preprocess_datasets, #Функция, которая применяется ко всем строкам
    batched=True,        #Использовать батчинг
    num_proc=1,         #Количество процессов
    remove_columns=small_val_ds.column_names,  #Удаляем исходные колонки
    cache_file_name="./cdatasets/val_ds_short_1024.cache" #Папка с кешом предобработанных данных
)


In [9]:
tokenizer.batch_decode([train_ds_tokenize[0]['input_ids']],skip_special_tokens=True)[0]

'You are a strict AI assistant designed to answer a question based on context. Answer verbatim from the context. Do not guess or formulate, answer as it is in the context. If there is no answer in the context, answer "No answer".\nContext:\nBeyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time.\nQuestion:\nWhen did Beyonce start becoming popular?\nAnswer: '

In [10]:
# Check tokenized data
print(tokenizer.batch_decode([train_ds_tokenize[0]['input_ids']], skip_special_tokens=True))
print(tokenizer.batch_decode([val_ds_tokenize[0]['input_ids']], skip_special_tokens=True))
print(tokenizer.batch_decode([small_train_ds_tokenize[0]['input_ids']], skip_special_tokens=True))
print(tokenizer.batch_decode([small_val_ds_tokenize[0]['input_ids']], skip_special_tokens=True))

['You are a strict AI assistant designed to answer a question based on context. Answer verbatim from the context. Do not guess or formulate, answer as it is in the context. If there is no answer in the context, answer "No answer".\nContext:\nBeyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time.\nQuestion:\nWhen did Beyonce start becoming popular?\nAnswer: ']
['You are a strict AI assistant designed to answer a question based on context. Answer verbatim from the context. Do not guess or formulate, answer as it is in the context. If there is no answer in the context, answer "No answer".\nContext:\

#### Функция для формирования батчей (при загрузке из Dataloader'a)

In [11]:
def custom_collate_fn(batch):
    ''' 
    batch - список словарей ....
        {"input_ids": [...], "attention_mask": [...], "answers": "..."},
        {"input_ids": [...], "attention_mask": [...], "answers": "..."},
    '''
    input_ids = [torch.tensor(item['input_ids']) for item in batch]
    attention_mask = [torch.tensor(item['attention_mask']) for item in batch]
    labels = [torch.tensor(item['labels']) for item in batch]
    answers = [item['answers'] for item in batch]

    return {
        "input_ids": torch.stack(input_ids),
        "attention_mask": torch.stack(attention_mask),
        "labels": torch.stack(labels),
        "answers": answers
    }

#### Dataloader's

In [12]:
train_dataloader = DataLoader(
    train_ds_tokenize,                # Датасет (например, tokenized_dataset)
    batch_size=BATCH_SIZE,         # Размер батча
    shuffle=False,         # Перемешивать данные
    num_workers=0,         # Количество потоков для загрузки
    collate_fn=custom_collate_fn,       # Функция для сборки батча
    pin_memory=False,      # Копировать данные в CUDA-память
    drop_last=False,       # Отбрасывать последний неполный батч
    prefetch_factor=None,     # Количество батчей для предварительной загрузки (сколько батчей будет загружено сразу для  ускорения, только в параллельном режиме (когда num_workers > 0)) (None - если не используем)
    persistent_workers=False  # Сохранять рабочие потоки между итерациями
)
val_dataloader = DataLoader(
    val_ds_tokenize,                # Датасет (например, tokenized_dataset)
    batch_size=BATCH_SIZE_EVAL,         # Размер батча
    shuffle=False,         # Перемешивать данные
    num_workers=0,         # Количество потоков для загрузки
    collate_fn=custom_collate_fn,       # Функция для сборки батча
    pin_memory=False,      # Копировать данные в CUDA-память
    drop_last=False,       # Отбрасывать последний неполный батч
    prefetch_factor=None,     # Количество батчей для предварительной загрузки (сколько батчей будет загружено сразу для  ускорения, только в параллельном режиме (когда num_workers > 0)) (None - если не используем)
    persistent_workers=False  # Сохранять рабочие потоки между итерациями
)


small_train_ds_tokenize_dataloader = DataLoader(
    small_train_ds_tokenize,                # Датасет (например, tokenized_dataset)
    batch_size=BATCH_SIZE_EVAL,         # Размер батча
    shuffle=False,         # Перемешивать данные
    num_workers=0,         # Количество потоков для загрузки
    collate_fn=custom_collate_fn,       # Функция для сборки батча
    pin_memory=False,      # Копировать данные в CUDA-память
    drop_last=False,       # Отбрасывать последний неполный батч
    prefetch_factor=None,     # Количество батчей для предварительной загрузки (сколько батчей будет загружено сразу для  ускорения, только в параллельном режиме (когда num_workers > 0)) (None - если не используем)
    persistent_workers=False  # Сохранять рабочие потоки между итерациями
)

small_val_ds_tokenize_dataloader = DataLoader(
    small_val_ds_tokenize,                # Датасет (например, tokenized_dataset)
    batch_size=BATCH_SIZE_EVAL,         # Размер батча
    shuffle=False,         # Перемешивать данные
    num_workers=0,         # Количество потоков для загрузки
    collate_fn=custom_collate_fn,       # Функция для сборки батча
    pin_memory=False,      # Копировать данные в CUDA-память
    drop_last=False,       # Отбрасывать последний неполный батч
    prefetch_factor=None,     # Количество батчей для предварительной загрузки (сколько батчей будет загружено сразу для  ускорения, только в параллельном режиме (когда num_workers > 0)) (None - если не используем)
    persistent_workers=False  # Сохранять рабочие потоки между итерациями
)

print(
    len(train_dataloader), #Количество батчей
    len(val_dataloader), #Количество батчей
    len(small_train_ds_tokenize_dataloader),
    len(small_val_ds_tokenize_dataloader)
)

65053 5930 16 16


### Функции для оценки ответов (нормализация текста, EM, F1)

In [13]:
def normalize_text(text):
    """
    Нормализует текст: нижний регистр, удаление пробелов и пунктуации
    """
    text = text.lower() # Приводим к нижнему регистру
    text = text.translate(str.maketrans("", "", string.punctuation)) # Удаляем пунктуацию
    text = re.sub(r'\s+', ' ', text).strip() # Удаляем лишние пробелы
    return text

def compute_exact_match(prediction, ground_truth):
    """
    Вычисляет Exact Match для одного примера
    Принимает 2 строки для сравнения
    """
    return int(normalize_text(prediction) == normalize_text(ground_truth))

def compute_f1_score(prediction, ground_truth):
    """
    Вычисляет F1 Score для одного примера
    Принимает 2 строки для сравнения
    """
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(ground_truth).split()
    
    if len(pred_tokens) == 0 and len(truth_tokens) == 0: # Если оба ответа пустые, F1 = 1
        return 1.0
    if len(pred_tokens) == 0 or len(truth_tokens) == 0: # Если один из ответов пустой, F1 = 0
        return 0.0
    
    # Находим общие токены
    common_tokens = set(pred_tokens) & set(truth_tokens)
    tp = len(common_tokens)
    
    precision = tp / len(pred_tokens)  # Precision = TP / (TP + FP), где FP = предсказанные токены, не входящие в правильные
    recall = tp / len(truth_tokens) # Recall = TP / (TP + FN), где FN = правильные токены, не входящие в предсказанные
    
    # F1 = 2 * (precision * recall) / (precision + recall)
    if precision + recall == 0:
        return 0.0
    return 2 * (precision * recall) / (precision + recall)

#### Вычисление средних метрик по всему датасету

In [14]:
def calculate_evaluate_metrics_EM_F1(em_all, f1_all):
    ''' 
    Функция для вычисления среднего значения EM и F1
    '''
    em_all = np.array(em_all)
    f1_all = np.array(f1_all)
    print(f"EM: {em_all.mean()}, F1: {f1_all.mean()}")
    return em_all.mean(), f1_all.mean()

### Функция для оценки ответов модели

* Расчет метрик от HF закоментированны намеренно.

In [15]:
def eval_model_ds(model, tokenizer, dataloader, logger = None, split_seq = "\nAnswer: "):

    em_all = []
    f1_all = []

    # ---- Для вычисления метрик от HF ----
    # pred = []
    # ref = []

    cnt_index = 0
    for idx, batch in enumerate(dataloader):
        inputs_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels'] #Только для обучения 
        answers = batch['answers'] #Список ответов на естественном языке

        # ---- Перенести тензоры на DEVICE ----
        # ---- Генерация ответа моделью ----
        with torch.no_grad(): #Отключить накопление градиентов (во время инференса)
            output_model = model.generate(
                input_ids=inputs_ids.to(DEVICE),
                attention_mask=attention_mask.to(DEVICE),
                max_new_tokens=MAX_LEN_LABELS_AND_NEW_TOKENS,
                temperature=TEMPERATURE
            ) #tensor: input_ids - (batch_size, max_len + max_new_tokens)
        
        # ---- Декодирование ----
        text_output_model = tokenizer.batch_decode(output_model, skip_special_tokens=True)
        #  ---- Декодирование исходной последовательности ----
        input_decode = tokenizer.batch_decode(inputs_ids, skip_special_tokens=True)

        # ---- Пробегаемся по ответам модели и вычисляем метрики ----
        for text_item, answr, inp_seq in zip(text_output_model, answers, input_decode):
            answer_model_text = ""
            text_split = text_item.split(split_seq)
            # if len(text_split) > 1:
            answer_model_text = text_split[-1].strip()
            answer_model_text = "" if answer_model_text == "No answer" else answer_model_text

            em_item = compute_exact_match(answer_model_text, answr)
            f1_item = compute_f1_score(answer_model_text, answr)

            em_all.append(em_item)
            f1_all.append(f1_item)
            
            # ---- Формирование списков для вычисления метрик от HF ----
            # pred.append({"id" : f"{cnt_index}", "prediction_text" : answer_model_text, "no_answer_probability" : 1.0 if answer_model_text == "" else 0.0})
            # ref.append({"id": f"{cnt_index}", "answers": {"text": [answr], "answer_start": []}})
            # cnt_index += 1

            if logger:
                logger.info(f"-----\n{inp_seq}\n[ANSW]:{answr}\n[MODL]:{answer_model_text}\nEM:{em_item}\nF1:{f1_item}\n-----")
        
        # ---- Очистка памяти ----
        del inputs_ids, attention_mask, answers, output_model
        torch.cuda.empty_cache()
        gc.collect()
    
    # ---- Вычисление метрик HF ----
    # hg_res = squad_metric.compute(predictions=pred, references=ref)
    # if logger:
    #     logger.info(f"HF EM: {hg_res['exact']} F1: {hg_res['f1']}")
    
    return em_all, f1_all

In [16]:
hh = "     \n \n \n No answer    "
hh = hh.strip()
hh

'No answer'

### Для тонкой настройки

In [17]:
#Функция для вычисления потерь на validation
def evaluate(model, dataloader):
    ''' 
    Вычисляем loss на валидационном датасете 
    dataloader : tqdm
    '''
    model.attn_implementation="sdpa"
    model.eval() #Переводим модель в режим оценки
    total_loss_eval = 0
    for idx, batch in enumerate(dataloader):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        
        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        total_loss_eval += outputs.loss.item()
       
        #Чистим: outputs, input_ids, attention_mask, labels для сохранения памяти
        del outputs, input_ids, attention_mask, labels
        torch.cuda.empty_cache()
        gc.collect()

    return total_loss_eval / len(dataloader)

#### Функция для FT

In [18]:
def my_train(model, dataloader, dataloader_val, optimizer, epochs = 1, accumulation_steps = 8, logger = None, train_logg_step = 100, dir_save_model = "./default_ft"):
    ''' Функция для тренировки модели '''
    GLOBAL_STEP = 0
    TOTAL_LOSS = 0
    loss_for_logging = 0 
    steps_for_logging = 0

    for epoch in range(epochs):
        tqdm_dataloader = tqdm(dataloader, desc=f"Train EPOCH: {epoch+1}")
        total_loss_epoch = 0
                        
        model.attn_implementation="eager"
        model.train()
        
        # Очищаем градиенты до захода в цикл
        optimizer.zero_grad()

        for idx, batch in enumerate(tqdm_dataloader): #Идем по даталоадеру
            #Распаковываем batch
            input_ids = batch['input_ids']
            attention_mask =  batch['attention_mask']
            labels = batch['labels']
            #Переносим на устройство
            input_ids, attention_mask, labels = input_ids.to(DEVICE), attention_mask.to(DEVICE), labels.to(DEVICE)
            
            # Размеры input_ids и labels должны совпадать, для вычисления loss.
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            loss = outputs.loss #Получаем потери
            loss = loss / accumulation_steps #Нормализация для накопления
            loss.backward() #Обратное распространение

            # Сохраняем потерю
            batch_loss = loss.item() * accumulation_steps
            total_loss_epoch += batch_loss
            TOTAL_LOSS += batch_loss
            loss_for_logging += batch_loss
            steps_for_logging += 1

            #Шаг оптимизатора только после накопления градиентов
            if (idx + 1) % accumulation_steps == 0 or (idx + 1) == len(tqdm_dataloader):
                optimizer.step() #Обновление параметров (Градиенты накапливаются до вызова optimizer.step())
                optimizer.zero_grad()  ## Очищаем градиенты после шага оптимизатора

            if GLOBAL_STEP > 0 and GLOBAL_STEP % train_logg_step == 0:
                avg_loss = loss_for_logging / steps_for_logging
                if logger: 
                    logger.info(f"Step {GLOBAL_STEP}, Avg Train Loss (last {train_logg_step} steps): {avg_loss:.4f}")
                print(f"Step {GLOBAL_STEP}, Avg Train Loss (last {train_logg_step} steps): {avg_loss:.4f}")
                # Сбрасываем накопленные значения
                loss_for_logging = 0
                steps_for_logging = 0
            
            GLOBAL_STEP += 1

            #Чистим: outputs, input_ids, attention_mask, labels для сохранения памяти
            del outputs, input_ids, attention_mask, labels, loss
            torch.cuda.empty_cache()
            gc.collect()

        #Средняя потеря за эпоху
        avg_loss_epoch = total_loss_epoch / len(tqdm_dataloader)
        if logger:
            logger.info(f"EPOCH {epoch+1}, AVG TRAIN LOSS: {avg_loss_epoch:.4f}")
        print(f"EPOCH {epoch+1}, AVG TRAIN LOSS: {avg_loss_epoch:.4f}")

        #Оценка на валидационной выборке
        tqdm_dataloader_val = tqdm(dataloader_val, desc=f"Val EPOCH: {epoch+1}")
        val_loss = evaluate(model, tqdm_dataloader_val)
        if logger:
            logger.info(f"EPOCH {epoch+1}, Val loss: {val_loss:.4f}")
        print(f"EPOCH {epoch+1}, Val loss: {val_loss:.4f}")

        #Оценка метрик EM и F1 (На валидационном датасете)
        em_all, f1_all = eval_model_ds(model, tokenizer, dataloader_val, logger=logger)
        em_r, f1_r = calculate_evaluate_metrics_EM_F1(em_all, f1_all)
        if logger:
            logger.info(f"EPOCH {epoch+1}, EM: {em_r} F1: {f1_r}")
        print(f"EPOCH {epoch+1}, EM: {em_r} F1: {f1_r}")

        # Сохранение модели (опционально)
        model.save_pretrained(f"{dir_save_model}/model_epoch_{epoch+1}")
        tokenizer.save_pretrained(f"{dir_save_model}/model_epoch_{epoch+1}")

    if logger:
        logger.info("Training completed!")
    print("Done!")

### LoRA

In [19]:
# -------- Параметры для LORA --------
LORA_R = 32                       # Ранг LoRA (Размер матриц адаптации)
lora_alpha = 2*LORA_R                  # Масштабирующий коэффициент (Уселение эффекта адаптации)
target_modules = ["q_proj", "v_proj"]   # Модули для LoRA (Применяется к слоям внимания)
#q_proj, k_proj, v_proj, o_proj

In [20]:
# -------- LORA Config --------
lora_config = LoraConfig(
    r=LORA_R,                       # Ранг LoRA
    lora_alpha=lora_alpha,          # Масштабирование
    target_modules=target_modules,  # Целевые модули
    lora_dropout=0.05,              # Dropout для регуляризации
    bias="none",                    # Без смещений в LoRA
    task_type="CAUSAL_LM",          # Тип задачи
)



In [21]:
LoRA_model = get_peft_model(model, lora_config)

In [22]:
#Сколько параметров будет обучаться при LoRA
trainable_params_LoRa = sum(p.numel() for p in LoRA_model.parameters() if p.requires_grad)
print(f"Trainable parameters: {trainable_params_LoRa:,}")

Trainable parameters: 2,981,888


### Optimizer for LoRA

In [23]:
optimizer = AdamW(LoRA_model.parameters(), lr=LEARNING_RATE, betas=BETAS, eps=EPS)

In [24]:
# --- Создаем файл логга ---
timestamp = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
log_filename = f"{EVAL_MODEL_DIR_OUT}/model_train_lora_256_{timestamp}.log"

logging.basicConfig(
    level=logging.INFO,  # Уровень логирования
    format='%(asctime)s - %(levelname)s - %(message)s',  # Формат сообщений
    handlers=[
        logging.FileHandler(log_filename, encoding='utf-8'),  # Запись в файл
        # logging.StreamHandler()  # Вывод в консоль (опционально)
    ]
)

my_logger = logging.getLogger()

my_logger.info(f"""
Title: This step train model with use LoRA
Process: Train model

--- Parameters Training ---
MODEL: {MODEL_PATH}
DEVICE: {DEVICE}
max_len_tokens: {MAX_LEN_PROMPT_TOKENIZER}
new_max_tokens: {MAX_LEN_LABELS_AND_NEW_TOKENS}
TEMPERATURE: {TEMPERATURE}

BATCH_SIZE: {BATCH_SIZE}
BATCH_SIZE_EVAL: {BATCH_SIZE_EVAL}

PADDING_SIDE_TOKENIZER: {PADDING_SIDE_TOKENIZER}

OUT_DATA_CACHE: {OUT_DATA_CACHE}
CACHE_DIR: {CACHE_DIR}
EVAL_MODEL_DIR_OUT: {EVAL_MODEL_DIR_OUT}

Train data: {PATH2DATA_TRAIN}
Val data: {PATH2DATA_VAL}

--- Parameters for LORA ---
TRAINED PARAMETERS: {trainable_params_LoRa}

EPOCH: {EPOCH}
LEARNING_RATE: {LEARNING_RATE}
GRADIENT_ACCUMULATION_STEPS: {GRADIENT_ACCUMULATION_STEPS}
TRAIN_LOGG_STEP: {TRAIN_LOGG_STEP}
SAVE_MODEL_DIR_LORA: {SAVE_MODEL_DIR_LORA}
SAVE_MODEL_DIR_DORA: {SAVE_MODEL_DIR_DORA}

--- Parameters for optimizer ---
BETAS: {BETAS}
EPS: {EPS}

--- Model architecture ---
{str(LoRA_model)}

--- Model config ---
{LoRA_model.config}
""")

### Запуск для LoRA

In [25]:
# Full
# my_train(
#     LoRA_model, 
#     train_dataloader, 
#     val_dataloader, 
#     optimizer, 
#     epochs = EPOCH, 
#     accumulation_steps = GRADIENT_ACCUMULATION_STEPS, 
#     logger = my_logger, 
#     train_logg_step = TRAIN_LOGG_STEP, 
#     dir_save_model = SAVE_MODEL_DIR_LORA
# )

# Short 
my_train(
    LoRA_model, 
    small_train_ds_tokenize_dataloader, 
    small_val_ds_tokenize_dataloader, 
    optimizer, 
    epochs = EPOCH, 
    accumulation_steps = GRADIENT_ACCUMULATION_STEPS, 
    logger = my_logger, 
    train_logg_step = TRAIN_LOGG_STEP, 
    dir_save_model = SAVE_MODEL_DIR_LORA
)

Train EPOCH: 1:   0%|          | 0/16 [00:00<?, ?it/s]

Train EPOCH: 1: 100%|██████████| 16/16 [00:26<00:00,  1.66s/it]


EPOCH 1, AVG TRAIN LOSS: 13.8542


Val EPOCH: 1: 100%|██████████| 16/16 [00:07<00:00,  2.09it/s]


EPOCH 1, Val loss: 10.9801
EM: 0.25, F1: 0.319612678987679
EPOCH 1, EM: 0.25 F1: 0.319612678987679
Done!


Потеря на обучении начала расти, возможно только в рамках одной эпохе, дальше пошла бы на спад, касаемое потери на ваидации, нет возможности определить будет расти или падать, так как запускается раз в эпоху. Низкие значения метрики EM и F1 - изменил промпт, а в функции обрезки ответа модели для оценки не изменил, следовательно, функция работала некорректно, что можно заметить, по очень низким метриками (в логах так же можно это заметить).

Даже если запустить в качестве теста на маленьких датасетах, F1 сохраняется на уровня 0.1 - 0.2.  

Стоило пофиксить функцию оценки, как на коротких датасетах (выборка первых 32 элементов) метрики подрасли до 0.25 и 0.31 EM and F1 соответственно.

### Перед DoRA

In [26]:
del LoRA_model
torch.cuda.empty_cache()
gc.collect()

0

### DoRA

In [27]:
# -------- Параметры для DORA --------
dora_config = DoraConfig(
    r=LORA_R,
    lora_alpha=lora_alpha,
    target_modules=target_modules,
    lora_dropout=0.05,  # Дропаут
    bias="none",  # Без смещения
    task_type="CAUSAL_LM",  # Тип задачи
    dora_simple=True,  # Упрощённый режим DoRA (Если False, норма вычисляется без .detach(), что позволяет градиентам распространяться).
    Wdecompose_target_modules=["q_proj", "v_proj"]  # Модули для декомпозиции весов
)

In [28]:
# Добавить адаптеры к исходной модели (Не спутать с моделью, после применения конфига на прошлом шаге) 
DoRA_model = get_peft_model(model, dora_config)

model.layers.0.self_attn.q_proj.weight_m_wdecomp.weight is trainable
model.layers.0.self_attn.q_proj.lora_A.weight is trainable
model.layers.0.self_attn.q_proj.lora_B.weight is trainable
model.layers.0.self_attn.v_proj.weight_m_wdecomp.weight is trainable
model.layers.0.self_attn.v_proj.lora_A.weight is trainable
model.layers.0.self_attn.v_proj.lora_B.weight is trainable
model.layers.1.self_attn.q_proj.weight_m_wdecomp.weight is trainable
model.layers.1.self_attn.q_proj.lora_A.weight is trainable
model.layers.1.self_attn.q_proj.lora_B.weight is trainable
model.layers.1.self_attn.v_proj.weight_m_wdecomp.weight is trainable
model.layers.1.self_attn.v_proj.lora_A.weight is trainable
model.layers.1.self_attn.v_proj.lora_B.weight is trainable
model.layers.2.self_attn.q_proj.weight_m_wdecomp.weight is trainable
model.layers.2.self_attn.q_proj.lora_A.weight is trainable
model.layers.2.self_attn.q_proj.lora_B.weight is trainable
model.layers.2.self_attn.v_proj.weight_m_wdecomp.weight is traina

### optimizer for DoRA

In [29]:
optimizerDora = AdamW(DoRA_model.parameters(), lr=LEARNING_RATE, betas=BETAS, eps=EPS)

In [30]:
#Сколько параметров будет обучаться при DoRA
trainable_params_Dora = sum(p.numel() for p in DoRA_model.parameters() if p.requires_grad)
print(f"Trainable parameters: {trainable_params_Dora:,}")

Trainable parameters: 3,015,168


In [31]:
# --- Создаем файл логга ---
timestamp = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
log_filename = f"{EVAL_MODEL_DIR_OUT}/model_train_dora_256_{timestamp}.log"

logging.getLogger().handlers.clear() #Очистка предыдущего

logging.basicConfig(
    level=logging.INFO,  # Уровень логирования
    format='%(asctime)s - %(levelname)s - %(message)s',  # Формат сообщений
    handlers=[
        logging.FileHandler(log_filename, encoding='utf-8'),  # Запись в файл
        # logging.StreamHandler()  # Вывод в консоль (опционально)
    ]
)

my_logger2 = logging.getLogger()

my_logger2.info(f"""
Title: This step train model with use DoRA
Process: Train model

--- Parameters Training ---
MODEL: {MODEL_PATH}
DEVICE: {DEVICE}
max_len_tokens: {MAX_LEN_PROMPT_TOKENIZER}
new_max_tokens: {MAX_LEN_LABELS_AND_NEW_TOKENS}
TEMPERATURE: {TEMPERATURE}

BATCH_SIZE: {BATCH_SIZE}
BATCH_SIZE_EVAL: {BATCH_SIZE_EVAL}

PADDING_SIDE_TOKENIZER: {PADDING_SIDE_TOKENIZER}

OUT_DATA_CACHE: {OUT_DATA_CACHE}
CACHE_DIR: {CACHE_DIR}
EVAL_MODEL_DIR_OUT: {EVAL_MODEL_DIR_OUT}

Train data: {PATH2DATA_TRAIN}
Val data: {PATH2DATA_VAL}

--- Parameters for DORA ---
TRAINED PARAMETERS: {trainable_params_Dora}

EPOCH: {EPOCH}
LEARNING_RATE: {LEARNING_RATE}
GRADIENT_ACCUMULATION_STEPS: {GRADIENT_ACCUMULATION_STEPS}
TRAIN_LOGG_STEP: {TRAIN_LOGG_STEP}
SAVE_MODEL_DIR_LORA: {SAVE_MODEL_DIR_LORA}
SAVE_MODEL_DIR_DORA: {SAVE_MODEL_DIR_DORA}

--- Parameters for optimizer ---
BETAS: {BETAS}
EPS: {EPS}

--- Model architecture ---
{str(DoRA_model)}

--- Model config ---
{DoRA_model.config}
""")

In [32]:
# Full
# my_train(
#     DoRA_model, 
#     train_dataloader, 
#     val_dataloader, 
#     optimizer, 
#     epochs = EPOCH, 
#     accumulation_steps = GRADIENT_ACCUMULATION_STEPS, 
#     logger = my_logger2, 
#     train_logg_step = TRAIN_LOGG_STEP, 
#     dir_save_model = SAVE_MODEL_DIR_DORA
# )

# Short 
my_train(
    DoRA_model, 
    small_train_ds_tokenize_dataloader, 
    small_val_ds_tokenize_dataloader, 
    optimizerDora, 
    epochs = EPOCH, 
    accumulation_steps = GRADIENT_ACCUMULATION_STEPS, 
    logger = my_logger2, 
    train_logg_step = TRAIN_LOGG_STEP, 
    dir_save_model = SAVE_MODEL_DIR_DORA
)

Train EPOCH: 1: 100%|██████████| 16/16 [00:25<00:00,  1.62s/it]


EPOCH 1, AVG TRAIN LOSS: 13.7965


Val EPOCH: 1: 100%|██████████| 16/16 [00:07<00:00,  2.22it/s]


EPOCH 1, Val loss: 10.9331
EM: 0.25, F1: 0.3300007284382284
EPOCH 1, EM: 0.25 F1: 0.3300007284382284
Done!


Про увеличение метрик EM и F1 можно сказать и тут. То есть проблема заключалась в расчете метрик, а не в самой модели.